**How to Query the Google Analytics Sample Dataset (BigQuery)**

In [1]:
import bq_helper
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
google_analytics = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="data:google_analytics_sample")

In [2]:
bq_assistant = BigQueryHelper("bigquery-public-data", "google_analytics_sample")
bq_assistant.list_tables()

['ga_sessions_20160801',
 'ga_sessions_20160802',
 'ga_sessions_20160803',
 'ga_sessions_20160804',
 'ga_sessions_20160805',
 'ga_sessions_20160806',
 'ga_sessions_20160807',
 'ga_sessions_20160808',
 'ga_sessions_20160809',
 'ga_sessions_20160810',
 'ga_sessions_20160811',
 'ga_sessions_20160812',
 'ga_sessions_20160813',
 'ga_sessions_20160814',
 'ga_sessions_20160815',
 'ga_sessions_20160816',
 'ga_sessions_20160817',
 'ga_sessions_20160818',
 'ga_sessions_20160819',
 'ga_sessions_20160820',
 'ga_sessions_20160821',
 'ga_sessions_20160822',
 'ga_sessions_20160823',
 'ga_sessions_20160824',
 'ga_sessions_20160825',
 'ga_sessions_20160826',
 'ga_sessions_20160827',
 'ga_sessions_20160828',
 'ga_sessions_20160829',
 'ga_sessions_20160830',
 'ga_sessions_20160831',
 'ga_sessions_20160901',
 'ga_sessions_20160902',
 'ga_sessions_20160903',
 'ga_sessions_20160904',
 'ga_sessions_20160905',
 'ga_sessions_20160906',
 'ga_sessions_20160907',
 'ga_sessions_20160908',
 'ga_sessions_20160909',


In [3]:
bq_assistant.head("ga_sessions_20160801", num_rows=3)

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,None,1,1470117657,1470117657,20160801,"{'visits': 1, 'hits': 3, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 23, 'minu...",7194065619159478122,None,Social,Not Socially Engaged
1,None,151,1470083489,1470083489,20160801,"{'visits': 1, 'hits': 3, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",8159312408158297118,None,Social,Not Socially Engaged
2,None,1,1470052694,1470052694,20160801,"{'visits': 1, 'hits': 4, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Southea...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 4, 'minut...",9236304747882138291,None,Social,Not Socially Engaged


In [4]:
bq_assistant.table_schema("ga_sessions_20160801")

[SchemaField('visitorId', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('visitNumber', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('visitId', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('visitStartTime', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('date', 'STRING', 'NULLABLE', None, ()),
 SchemaField('totals', 'RECORD', 'NULLABLE', None, (SchemaField('visits', 'INTEGER', 'NULLABLE', None, ()), SchemaField('hits', 'INTEGER', 'NULLABLE', None, ()), SchemaField('pageviews', 'INTEGER', 'NULLABLE', None, ()), SchemaField('timeOnSite', 'INTEGER', 'NULLABLE', None, ()), SchemaField('bounces', 'INTEGER', 'NULLABLE', None, ()), SchemaField('transactions', 'INTEGER', 'NULLABLE', None, ()), SchemaField('transactionRevenue', 'INTEGER', 'NULLABLE', None, ()), SchemaField('newVisits', 'INTEGER', 'NULLABLE', None, ()), SchemaField('screenviews', 'INTEGER', 'NULLABLE', None, ()), SchemaField('uniqueScreenviews', 'INTEGER', 'NULLABLE', None, ()), SchemaField('timeOnScreen', 'INTEGER', 'NULLAB

What is the total number of transactions generated per device browser in July 2017?

In [5]:
query1 = """SELECT
device.browser,
SUM ( totals.transactions ) AS total_transactions
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
GROUP BY
device.browser
ORDER BY
total_transactions DESC;
        """
response1 = google_analytics.query_to_pandas_safe(query1)
response1.head(10)

,browser,total_transactions
0,Chrome,2106.0
1,Safari,66.0
2,Firefox,13.0
3,Internet Explorer,9.0
4,Edge,3.0
5,Opera,2.0
6,Android Webview,NaN
7,Puffin,NaN
8,osee2unifiedRelease,NaN
9,BlackBerry,NaN


The real bounce rate is defined as the percentage of visits with a single pageview. What was the real bounce rate per traffic source?

In [6]:
query2 = """SELECT
source,
total_visits,
total_no_of_bounces,
( ( total_no_of_bounces / total_visits ) * 100 ) AS bounce_rate
FROM (
SELECT
trafficSource.source AS source,
COUNT ( trafficSource.source ) AS total_visits,
SUM ( totals.bounces ) AS total_no_of_bounces
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
GROUP BY
source )
ORDER BY
total_visits DESC;
        """
response2 = google_analytics.query_to_pandas_safe(query2)
response2.head(10)

,source,total_visits,total_no_of_bounces,bounce_rate
0,google,40961,20636.0,50.379629
1,(direct),13087,6923.0,52.899824
2,mall.googleplex.com,6940,1054.0,15.187320
3,youtube.com,6411,4241.0,66.151926
4,analytics.google.com,2047,1082.0,52.857841
5,gdeals.googleplex.com,2000,196.0,9.800000
6,Partners,1861,946.0,50.832886
7,sites.google.com,1332,258.0,19.369369
8,m.facebook.com,673,430.0,63.893016
9,google.com,383,183.0,47.780679


What is the average amount of money spent per session in July 2017?

In [7]:
query6 = """SELECT
( SUM(total_transactionrevenue_per_user) / SUM(total_visits_per_user) ) AS
avg_revenue_by_user_per_visit
FROM (
SELECT
fullVisitorId,
SUM( totals.visits ) AS total_visits_per_user,
SUM( totals.transactionRevenue ) AS total_transactionrevenue_per_user
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
AND
totals.visits > 0
AND totals.transactions >= 1
AND totals.transactionRevenue IS NOT NULL
GROUP BY
fullVisitorId );
        """
response6 = google_analytics.query_to_pandas_safe(query6, max_gb_scanned=10)
response6.head(10)

,avg_revenue_by_user_per_visit
0,1.716877e+08


What is the sequence of pages viewed?

In [8]:
query7 = """SELECT
fullVisitorId,
visitId,
visitNumber,
hits.hitNumber AS hitNumber,
hits.page.pagePath AS pagePath
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`,
UNNEST(hits) as hits
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
AND
hits.type="PAGE"
ORDER BY
fullVisitorId,
visitId,
visitNumber,
hitNumber;
        """
response7 = google_analytics.query_to_pandas_safe(query7, max_gb_scanned=10)
response7.head(10)

,fullVisitorId,visitId,visitNumber,hitNumber,pagePath
0,0000000334692759449,1501106611,1,1,/home
1,0000000334692759449,1501106611,1,2,/google+redesign/bags
2,0000000334692759449,1501106611,1,3,/google+redesign/bags
3,0000000334692759449,1501106611,1,4,/home
4,0000000334692759449,1501106611,1,5,/google+redesign/accessories/fun
5,0000000334692759449,1501106611,1,6,/google+redesign/shop+by+brand/youtube
6,0000000334692759449,1501106611,1,7,/google+redesign/accessories/sports++fitness
7,0000000334692759449,1501106611,1,8,/google+redesign/bags/water+bottles+and+tumblers
8,0000000334692759449,1501106611,1,9,/google+redesign/bags/more+bags
9,0000000334692759449,1501106611,1,10,/google+redesign/bags/backpacks/home
